# Generate structured cases for the following scenarios

1. 52 year old man comes to ED complaining of chest pain
2. 45 year old man comes to outpatient clinic complaining of bloody urine
3. 49 year old man comes to clinic complaining of back pain 
4. 28 year old man walks into ED after a motor vehicle accident
5. 32 year old woman walks into clinic with a positive pregnancy test
6. 11 year old girl comes to clinic with an elevated glucose test from school nurse
7. 69 year old man complains of left arm pain
8. 63 year old woman comes back to clinic for follow-up of diabetes
9. 28 year old lady comes to ED after sexual assault
10. 40 year old woman complains of right calf pain
11. 65 year old man presents with new onset hoarseness of voice
12. 70 year old lady presenting to clinic with neck pain
13. 55 year old female admitted to ED for abdominal pain
14. 38 year old woman presents to clinic with headache
15. 35 year old lady complains of irregular menses
16. 24 year old young female presents to clinic with regular pain during sex
17. 80 year old elderly gentleman complaining of hearing loss
18. 5 year old girl coming in to clinic with jaundice
19. 8 month old boy presents to ED with fever
20. 22 year old young gentleman presents to clinic with cough
21. 49 year old lady presenting to clinic with jaundice
22. 55 year old male complains of dizziness
23. 35 year old young lady presenting to the clinic with knee pain
24. 28 year old young man complains of heel pain
25. 19 month old girl presenting to clinic with fever
26. 58 year old woman complains of chronic cough
27. 59 year old man complains of chronic fatigue
28. 61 year old man comes to clinic for followup on hypertension
29. 23 year old young lady complains of sleeping issues
30. 2 year old girl presents to clinic with persistent noisy breathing
31. 24 year old woman presents to ED with abdominal pain
32. 68 year old elderly lady complaining of forgetfulness and disorientation
33. 49 year old man came to clinic complaining of fatigue
34. 32 year old young lady presents to clinic with fatigue
35. 26 year old man complains of visual and auditory hallucinations
36. 29 year old woman presents in clinic for pre employment physical
37. 59 year old man presents to ED with bloody stools
38. 61 year old man presents to clinic with tremors and walking problems
39. 32 year old young woman came to clinic with weight gain
40. 5 month old girl presents to ED with multiple episodes of diarrhoea
41. 7 year old boy presents to clinic with bed wetting
42. 13 month old girl presents to ED with seizures
43. 25 year old man complains of sore throat
44. 54 year old man presents to ED with fall and loss of consciousness

In [1]:
import pandas as pd

details = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Details")
history = pd.read_excel("Data/BaseCase.xlsx", sheet_name="History Taking", keep_default_na=False)
physical = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Physical Examination")
diagnosis = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Differential Diagnosis")
Investigations = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Investigations")

In [169]:
from importlib import reload
import simulator
reload(simulator.generators)

history_taking_example = "\n".join(
  [f"Question: {row['Question']} | Response: {row['Patient Response']}"
    for index, row in history.iterrows()])

case_generator = simulator.generators.CaseGenerator(
  model_name="gpt-4o",
  case_stem = "28 year old young man complains of heel pain",
  basic_details_example= details["Case Prompt"].values[0],
  vitals_example = details["Vitals"].values[0],
  physical_presentation_example = details["Physical presentation"].values[0],
  challenging_question_example = details["Challenging Questions to Ask"].values[0],
  history_taking_example = history_taking_example
)

In [170]:
output = case_generator.generate_basic_details()

In [171]:
output

['Emily Carter, a 35-year-old female, presents to the clinic complaining of knee pain.',
 'BP: 120/80 mm Hg\nTemp: 98.4°F (36.9°C)\nRR: 18/minute\nHR: 78/minute, regular',
 None,
 '"Could this be something serious like arthritis or will it go away on its own?"']

In [172]:
output = case_generator.generate_history()

In [173]:
output

HistoryTakingChecklist(ChecklistItems=[HistoryTakingChecklistItem(question='Chief complaint', response='Knee pain'), HistoryTakingChecklistItem(question='Onset', response='Started two days ago after a minor fall during a hiking trip'), HistoryTakingChecklistItem(question='Location', response='Right knee'), HistoryTakingChecklistItem(question='Quality', response='Aching and sometimes sharp when moving'), HistoryTakingChecklistItem(question='Severity on a scale of 1 to 10', response='5 out of 10'), HistoryTakingChecklistItem(question='Radiation', response='No radiation'), HistoryTakingChecklistItem(question='Alleviating factors', response='Rest and keeping the leg elevated'), HistoryTakingChecklistItem(question='Exacerbating factors', response='Walking, bending, or putting weight on the knee'), HistoryTakingChecklistItem(question='Associated symptoms', response='Slight swelling and stiffness'), HistoryTakingChecklistItem(question='Previous episodes of similar pain', response='No'), Histo

In [174]:
for item in output.ChecklistItems:
  print(item.question)
  print(item.response)
  print()

Chief complaint
Knee pain

Onset
Started two days ago after a minor fall during a hiking trip

Location
Right knee

Quality
Aching and sometimes sharp when moving

Severity on a scale of 1 to 10
5 out of 10

Radiation
No radiation

Alleviating factors
Rest and keeping the leg elevated

Exacerbating factors
Walking, bending, or putting weight on the knee

Associated symptoms
Slight swelling and stiffness

Previous episodes of similar pain
No

Current medications
Occasional ibuprofen for pain

Past medical history
No significant medical history

Past surgical history
None

Family history
No family history of joint problems

Occupation
Office worker

Exercise
Regularly hikes and does yoga

Alcohol use
Socially, about once a week

Illicit drugs
None

Tobacco history
Never smoked

Diet
Balanced diet, mostly home-cooked meals

Drug Allergies
None



In [175]:
output = case_generator.generate_physical_exam()

In [176]:
output

PhysicalExamChecklist(ChecklistItems=[PhysicalExamChecklistItem(technique='Inspection of the knee', justification='To visually assess for swelling, redness, or deformity in the right knee.'), PhysicalExamChecklistItem(technique='Palpation of the knee', justification='To check for warmth, tenderness, or any palpable abnormalities such as effusion or crepitus.'), PhysicalExamChecklistItem(technique='Range of motion assessment', justification='To evaluate the extent of movement and identify any limitations or pain during flexion and extension of the knee.'), PhysicalExamChecklistItem(technique='Lachman test', justification='To assess the integrity of the anterior cruciate ligament (ACL) which could be affected due to the fall.'), PhysicalExamChecklistItem(technique='McMurray test', justification='To evaluate for meniscal tears, which could be a result of the twisting injury during the fall.'), PhysicalExamChecklistItem(technique='Varus and Valgus stress tests', justification='To assess th

In [177]:
output = case_generator.generate_investigations()

In [178]:
output

InvestigationsChecklist(ChecklistItems=[InvestigationsChecklistItem(investigation='X-ray of the right knee', justification='To assess for any fractures or bone abnormalities following the fall.'), InvestigationsChecklistItem(investigation='MRI of the right knee', justification='To evaluate soft tissue structures such as ligaments, tendons, and menisci for any tears or injuries.'), InvestigationsChecklistItem(investigation='Ultrasound of the right knee', justification='To assess for any effusion or soft tissue abnormalities, especially if MRI is not immediately available.'), InvestigationsChecklistItem(investigation='Blood tests (CBC, ESR, CRP)', justification='To rule out any underlying inflammatory or infectious process contributing to the knee pain and swelling.')])

In [179]:
output = case_generator.generate_ddx()

In [180]:
output

DdxChecklist(ChecklistItems=[DdxChecklistItem(diagnosis='Knee Sprain or Strain', justification='The patient reports knee pain following a minor fall, with symptoms of aching and sharp pain upon movement, as well as slight swelling and stiffness. These are common symptoms of a knee sprain or strain, which can occur due to overstretching or tearing of ligaments or muscles around the knee.'), DdxChecklistItem(diagnosis='Meniscal Tear', justification='The mechanism of injury (a fall during hiking) and the symptoms of pain, swelling, and stiffness suggest the possibility of a meniscal tear, which is a common knee injury that can occur with twisting or impact.'), DdxChecklistItem(diagnosis='Patellar Tendinitis', justification="The patient's regular hiking and yoga activities, combined with the recent fall, could lead to patellar tendinitis, characterized by pain and discomfort in the knee, especially when bending or putting weight on it."), DdxChecklistItem(diagnosis='Knee Contusion', justif

In [181]:
case_generator.export_case(file_name="23")

>>> Exporting basic details
>>> Exporting history
>>> Exporting Physical Exam
>>> Exporting Investigations
>>> Exporting Differential Diagnosis
